In [1]:
import os
import numpy as np
import efficientnet.tfkeras
from tensorflow.keras.models import load_model
from tensorflow.keras import models
import tensorflow as tf
from focal_loss import BinaryFocalLoss

In [2]:
os.environ["CUDA_VISIBLE_DEVICES"]="0"

# Load Network 15AB

In [3]:
model_dir ='/media/tohn/SSD/ModelTrainByImages/R2_1/models/B5R2_block5_15AB_1FC_3.h5' #model
model = load_model(model_dir)
height = width = model.input_shape[1]

x = model.get_layer('head_pooling').output
prediction_layer = model.output
model2 = models.Model(inputs= model.input, outputs=[prediction_layer,x]) 

In [4]:
# validation
import pandas as pd
base_dir  = '/media/tohn/SSD/Images/Image/'#แก้
dataframe = pd.read_csv( '/home/yupaporn/CCA_Project/Dataset_CCA_case_Testset1312.csv')# ---------------------------------------*เปลี่ยนไฟล์เเล้ว
validation_dir = os.path.join(base_dir, 'test')

#Train
train_df = pd.read_csv( '/home/yupaporn/CCA_Project/Dataset_CCA_case_Trainset5257.csv')# ---------------------------------------*เปลี่ยนไฟล์เเล้ว
base_dir0 = '/media/tohn/SSD/Images/Image/'#แก้
os.chdir(base_dir0)
train_dir = os.path.join(base_dir0, 'train')

In [5]:
import pandas as pd
df0 = pd.read_csv (r'/home/yupaporn/CCA_Project/Dataset_CCA_case_Trainset5257.csv')# ---------------------------------------*เปลี่ยนไฟล์เเล้ว
print(df0 .shape)
dataframe = df0[(df0['Path Crop']!='None' )&(df0['Path Crop']!='Nan')]
print(dataframe.shape)
print('Normal: ',dataframe[dataframe['Class']=='Normal'].shape)
print('Abnormal: ',dataframe[dataframe['Class']=='Abnormal'].shape)
dataframe.head(5)

(5257, 24)
(5257, 24)
Normal:  (3434, 24)
Abnormal:  (1823, 24)


,Unnamed: 0,Case,Abs Position,Sub Position,Class,Path Full,Path Crop,Views,fold,15AB,...,height,Rleft,Rtop,Rwidth,Rheight,Sub_class_New,tag_AjNit,tag_AjWan,Test150,Spilt
0,0,AB01_15,P3,P31,Abnormal,/media/tohn/HDD/VISION_dataset/USAI/ABnormal01...,/media/tohn/HDD/VISION_dataset/USAI/ABnormal01...,FP-B,3,MildFattyLiver,...,584.0,0.036832,0.107041,0.712707,0.649402,AB01,NaN,NaN,False,Train
1,1,AB01_15,P1,P1,Abnormal,/media/tohn/HDD/VISION_dataset/USAI/ABnormal01...,/media/tohn/HDD/VISION_dataset/USAI/ABnormal01...,FP-A,3,MildFattyLiver,...,614.0,0.143898,0.107041,0.595628,0.683267,AB01,NaN,NaN,False,Train
2,2,AB01_15,P2,P2,Abnormal,/media/tohn/HDD/VISION_dataset/USAI/ABnormal01...,/media/tohn/HDD/VISION_dataset/USAI/ABnormal01...,FP-A,3,MildFattyLiver,...,451.0,0.245009,0.113017,0.497278,0.501992,AB01,NaN,NaN,False,Train
3,3,AB01_15,P4,P42,Abnormal,/media/tohn/HDD/VISION_dataset/USAI/ABnormal01...,/media/tohn/HDD/VISION_dataset/USAI/ABnormal01...,FP-B,3,MildFattyLiver,...,611.0,0.155235,0.122977,0.718412,0.679283,AB01,NaN,NaN,False,Train
4,4,AB01_15,P4,P41,Abnormal,/media/tohn/HDD/VISION_dataset/USAI/ABnormal01...,/media/tohn/HDD/VISION_dataset/USAI/ABnormal01...,FP-B,3,MildFattyLiver,...,519.0,0.152057,0.162817,0.608229,0.577689,AB01,NaN,NaN,False,Train


In [6]:
batch_size = 64
epochs = 10

from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(
      rescale=1./255,
      rotation_range=30,
      width_shift_range=0.2,
      height_shift_range=0.2,
      brightness_range=[0.5,1.5],
      shear_range=0.4,
      zoom_range=0.2,
      horizontal_flip=False,
      fill_mode='nearest')

train_generator = train_datagen.flow_from_dataframe(
        dataframe = dataframe,
        directory = train_dir,
        x_col = 'Path Crop',
        y_col = 'Sub_class_New',# ---------------------------------------*เปลี่ยนไฟล์เเล้ว
        target_size = (height, width),
        batch_size=batch_size,
        color_mode= 'rgb',
        class_mode='categorical')

#label
labels = (train_generator.class_indices)
labels = dict((v,k.replace("C","")) for k,v in labels.items())
print(labels)

Found 5257 validated image filenames belonging to 15 classes.
{0: 'AB01', 1: 'AB02', 2: 'AB03', 3: 'AB04', 4: 'AB05', 5: 'AB06', 6: 'AB07', 7: 'AB081', 8: 'AB082', 9: 'AB083', 10: 'AB09', 11: 'AB10', 12: 'AB11', 13: 'AB12', 14: 'Normal'}


In [7]:
from tensorflow.keras.preprocessing import image
def predict_image(img_path):
    # Read the image and resize it
    img = image.load_img(img_path, target_size=(height, width))
    # Convert it to a Numpy array with target shape.
    x = image.img_to_array(img)
    # Reshape
    x = x.reshape((1,) + x.shape)
    x /= 255.
    result = model2.predict([x])
    result1 = result[0][0]
    result2 = result[1][0]

    return result1, result2

#Predict
pred_list = list()
prob_list = list()
vectorlist = list()
img_path=dataframe['Path Crop'].tolist()
for i in range(0,len(img_path)):
    result1,result2 = predict_image(img_path[i])
    result = np.argmax(result1)
    pred_list.append(labels[result])
    prob_list.append(result1[result])
    vectorlist.append(result2.tolist())
    
dataframe['category'] = pred_list
dataframe['Prob'] = prob_list

In [8]:
vector_array = np.array(vectorlist)
vector_array

array([[ 0.40253919,  0.18002251, -0.18678397, ..., -0.01196804,
         0.53766692, -0.07658342],
       [ 0.44914368,  0.28199571, -0.20333001, ...,  0.11549859,
         0.2241866 ,  0.03795064],
       [ 0.55702764,  0.25297821, -0.16948406, ..., -0.07377651,
         0.13611096,  0.04097907],
       ...,
       [ 0.71936274,  0.29354167, -0.18325303, ...,  0.41917655,
         0.05245944,  0.85134029],
       [ 0.05263624,  0.79944932, -0.04403901, ...,  0.81976956,
         0.15946686,  0.91900593],
       [ 0.06094814,  0.96647996, -0.11719888, ...,  0.76697558,
         0.28330526,  0.72980523]])

In [9]:
dataset = pd.DataFrame(vector_array[:, :])
dataset

,0,1,2,3,4,5,6,7,8,9,...,2038,2039,2040,2041,2042,2043,2044,2045,2046,2047
0,0.402539,0.180023,-0.186784,-0.154594,-0.020002,-0.182080,0.138693,0.165470,-0.174399,-0.185921,...,-0.171082,-0.121781,-0.071687,-0.068436,0.041892,-0.009122,0.091211,-0.011968,0.537667,-0.076583
1,0.449144,0.281996,-0.203330,-0.097550,0.053626,-0.172454,0.575487,0.205429,-0.145424,0.210573,...,-0.160589,-0.060622,0.070258,-0.145434,-0.151125,-0.059742,0.318514,0.115499,0.224187,0.037951
2,0.557028,0.252978,-0.169484,-0.195631,0.250338,-0.146651,0.189950,-0.130901,-0.167017,-0.046338,...,-0.178748,0.182256,0.047125,-0.091043,-0.083846,-0.189088,0.286967,-0.073777,0.136111,0.040979
3,0.000386,0.114845,0.007022,-0.066287,0.147522,-0.094377,-0.007889,-0.070110,-0.132005,-0.096130,...,-0.181015,-0.100758,0.239442,0.134625,0.081554,0.193782,-0.057682,0.011547,-0.053540,-0.028973
4,0.184263,0.029975,-0.205716,-0.168821,0.007270,-0.187991,0.274588,-0.052251,-0.136955,0.399787,...,-0.128447,-0.081682,-0.080243,-0.112200,0.078436,-0.173041,0.086912,0.251059,0.388342,-0.077735
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5252,0.876388,0.950660,-0.143525,-0.191516,0.266757,-0.007664,0.022300,0.460705,-0.066698,-0.144418,...,-0.116793,-0.175449,0.273187,0.123069,0.309657,0.031380,0.193792,0.304420,0.480895,0.636422
5253,1.009226,0.825011,0.040969,0.048938,0.603176,-0.010297,0.356882,0.390949,0.133900,-0.048576,...,0.138647,-0.094404,0.056603,0.396481,0.279164,-0.042167,0.238154,0.269945,-0.005251,0.770645
5254,0.719363,0.293542,-0.183253,-0.155861,0.199343,0.684893,-0.068810,0.330472,-0.067786,-0.035777,...,-0.100397,-0.147960,0.038633,0.882497,0.258186,0.576265,0.131403,0.419177,0.052459,0.851340
5255,0.052636,0.799449,-0.044039,-0.132899,0.806874,-0.208082,-0.007726,0.387321,-0.054545,0.403010,...,-0.079931,-0.211042,0.124955,0.702099,0.466079,0.047106,0.656479,0.819770,0.159467,0.919006


In [10]:
a = dataframe['Class'].tolist()
b = dataframe['Sub Position'].tolist()
c = dataframe['Views'].tolist()
d = dataframe['Sub_class_New'].tolist()
e= dataframe['Case'].tolist()
dataset['Case'] = e
dataset['Class'] = a 
dataset['SubPosition'] = b
dataset['Views'] = c
dataset['Sub_class_New'] = d

In [13]:
dataset

,0,1,2,3,4,5,6,7,8,9,...,2043,2044,2045,2046,2047,Case,Class,SubPosition,Views,Sub_class_New
0,0.402539,0.180023,-0.186784,-0.154594,-0.020002,-0.182080,0.138693,0.165470,-0.174399,-0.185921,...,-0.009122,0.091211,-0.011968,0.537667,-0.076583,AB01_15,Abnormal,P31,FP-B,AB01
1,0.449144,0.281996,-0.203330,-0.097550,0.053626,-0.172454,0.575487,0.205429,-0.145424,0.210573,...,-0.059742,0.318514,0.115499,0.224187,0.037951,AB01_15,Abnormal,P1,FP-A,AB01
2,0.557028,0.252978,-0.169484,-0.195631,0.250338,-0.146651,0.189950,-0.130901,-0.167017,-0.046338,...,-0.189088,0.286967,-0.073777,0.136111,0.040979,AB01_15,Abnormal,P2,FP-A,AB01
3,0.000386,0.114845,0.007022,-0.066287,0.147522,-0.094377,-0.007889,-0.070110,-0.132005,-0.096130,...,0.193782,-0.057682,0.011547,-0.053540,-0.028973,AB01_15,Abnormal,P42,FP-B,AB01
4,0.184263,0.029975,-0.205716,-0.168821,0.007270,-0.187991,0.274588,-0.052251,-0.136955,0.399787,...,-0.173041,0.086912,0.251059,0.388342,-0.077735,AB01_15,Abnormal,P41,FP-B,AB01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5252,0.876388,0.950660,-0.143525,-0.191516,0.266757,-0.007664,0.022300,0.460705,-0.066698,-0.144418,...,0.031380,0.193792,0.304420,0.480895,0.636422,344,Normal,P32,FP-B,Normal
5253,1.009226,0.825011,0.040969,0.048938,0.603176,-0.010297,0.356882,0.390949,0.133900,-0.048576,...,-0.042167,0.238154,0.269945,-0.005251,0.770645,344,Normal,P42,FP-B,Normal
5254,0.719363,0.293542,-0.183253,-0.155861,0.199343,0.684893,-0.068810,0.330472,-0.067786,-0.035777,...,0.576265,0.131403,0.419177,0.052459,0.851340,344,Normal,P52,FP-C,Normal
5255,0.052636,0.799449,-0.044039,-0.132899,0.806874,-0.208082,-0.007726,0.387321,-0.054545,0.403010,...,0.047106,0.656479,0.819770,0.159467,0.919006,344,Normal,P61,FP-C,Normal


In [12]:
os.chdir( '/home/yupaporn/CCA_Project')
testdf = dataset
# testdf.to_csv('FeatureVector_CCA_case_Trainset5257.csv') 